In [1]:
import polars as pl
import os

In [2]:
path_file= lambda x: os.path.join(os.getcwd(),'data',x)
df = pl.read_csv(path_file('marketing_data.csv'))
df.head(1)

ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
i64,i64,str,str,str,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
1826,1970,"""Graduation""","""Divorced""","""$84,835.00 """,0,0,"""6/16/14""",0,189,104,379,111,189,218,1,4,4,6,1,0,0,0,0,0,1,0,"""SP"""


In [3]:
df = df.with_columns(pl.col('Dt_Customer').str.strptime(pl.Date, "%m/%d/%y")) # covierte a fecha

In [4]:
df = df.rename({col:col.replace(' ','').replace('_','').lower() for col in df.columns}) # renombra la columna
df = df.with_columns(
    pl.col('income').str.replace_all(r'[$,]', '',)  # elimina el simbolo de dolar y la coma
    .str.replace_all(r' ', '')  # elimina los espacios
    .cast(pl.Float64)  # convierte a float
    .fill_null(0),  # rellena los valores nulos con 0
)
df.head(1)

id,yearbirth,education,maritalstatus,income,kidhome,teenhome,dtcustomer,recency,mntwines,mntfruits,mntmeatproducts,mntfishproducts,mntsweetproducts,mntgoldprods,numdealspurchases,numwebpurchases,numcatalogpurchases,numstorepurchases,numwebvisitsmonth,acceptedcmp3,acceptedcmp4,acceptedcmp5,acceptedcmp1,acceptedcmp2,response,complain,country
i64,i64,str,str,f64,i64,i64,date,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
1826,1970,"""Graduation""","""Divorced""",84835.0,0,0,2014-06-16,0,189,104,379,111,189,218,1,4,4,6,1,0,0,0,0,0,1,0,"""SP"""


# Cargar datos a google cloud 

In [6]:
#pip install gcsfs
import gcsfs

- **fs = gcsfs.GCSFileSystemEste :** Se crea un sistema de archivos virtual para trabajar con GCS usando la librería gcsfs. Este objeto permite leer y escribir archivos directamente en GCS como si fuera un sistema de archivos normal.

- **destino ='gs://data-etl-2025/datalake/datamkt.parquet'** Se define la ruta de destino en Google Cloud Storage donde se guardará el archivo Parquet.
    - data-etl-2025 es el nombre del bucket.

    - datalake/datamkt.parquet es la carpeta y nombre de archivo dentro del bucket.

In [ ]:
# Tenemos que definiar las credenciales de Google Cloud Storage
credentials = 'secretos/cursoetl-2025-afda877e10a1.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials
fs = gcsfs.GCSFileSystem() #Se crea un sistema de archivos virtual para trabajar con GCS usando la librería gcsfs
destino ='gs://data-etl-2025/datalake/datamkt.parquet' # boucket de destino
with fs.open(destino, 'wb') as f:
    df.write_parquet(f)  # escribe el dataframe en formato parquet 
